# Imports

In [23]:
import pandas as pd
import pandas_profiling
import numpy as np
from statsmodels.regression.linear_model import OLS, WLS
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from statsmodels.stats.diagnostic import linear_rainbow
from scipy.stats.stats import pearsonr
from scipy.stats import sem, t
from scipy import mean
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

In [2]:
df = pd.read_excel(r'data/ACQ360DataStacked_xlsx.xlsx')
df.head()

,ResponseID,Question,QuCode,QuCategory,QuTarget,QuType,Answer,Unnamed: 7
0,36,Last page,LastPg,Metadata,All,Autogenerated,2,NaN
1,36,Start language,StartLang,Metadata,All,Autogenerated,en,NaN
2,36,Date started,DateStarted,Metadata,All,Date,2016-06-07 15:46:10,NaN
3,36,Date last action,DateLastAction,Metadata,All,Date,2016-06-07 15:49:45,NaN
4,36,First 2 Characters of PIID,First2PIID,Attribute,All,Autogenerated,SS,NaN


In [3]:
#these are the questions whose responses will be IVs
cols = {'Were you part of an IPT (Integrated Procurement Team)-':'IPT_DUMMY',
        'Planning - How satisfied were you: [With the acquisition milestone schedule-]':'ACQMILE',
        'Planning - How satisfied were you: [With the procurement office’s ability to keep you informed of any changes to the acquisition milestone schedule-]':'SCHCHG',
        'Planning - How satisfied were you: [With the procurement office’s assistance in the Acquisition Plan process, which allowed you to better understand and participate in the procurement-]':'ACQASSIST',
        'Planning - How satisfied were you: [With the procurement office’s engagement with industry early in the acquisition process-]':'INDENG',
        'Communication - How satisfied were you: [With the procurement office’s responsiveness to your questions (communicating in a clear, courteous, timely, and professional manner)-]':'ACQCOM',
        'Communication - How satisfied were you: [With the procurement office’s effectiveness in resolving any issues or delays encountered during the acquisition process-]':'RESISS',
        'Communication - How satisfied were you: [With your understanding on how - and to whom – you should elevate problems for resolution-]':'ELEVPROB',
        'Communication - How satisfied were you: [With early communications describing the roles and responsibilities of the procurement office and of your office (program office)-]':'ACQRR',
        'Overall Satisfaction [How satisfied were you with your overall experience on this acquisition-]':'OVALLSAT'
        }

#filter for just the questions we're interested in
df = df[df['Question'].isin(cols)][['ResponseID','Question','Answer']]
df.head()

,ResponseID,Question,Answer
24,36,Were you part of an IPT (Integrated Procuremen...,NaN
62,71,Overall Satisfaction [How satisfied were you w...,5
64,71,Were you part of an IPT (Integrated Procuremen...,NaN
96,81,Overall Satisfaction [How satisfied were you w...,5
98,81,Were you part of an IPT (Integrated Procuremen...,NaN


In [4]:
#transform the data from long to wide, renaming columns to match names in EViews, treate a 
#dummy variable and create our design matrix

design_df = df.pivot(index = 'ResponseID', columns = 'Question', values = 'Answer').dropna()
design_df = design_df.rename(mapper = cols, axis = 1)
#make not being in an IPT the reference group
design_df['IPT_DUMMY'] = design_df['IPT_DUMMY'].map({'Yes':'yes', 'No':'no', 'No ':'no'})
design_df = design_df[cols.values()]
design_df.head()

Question,IPT_DUMMY,ACQMILE,SCHCHG,ACQASSIST,INDENG,ACQCOM,RESISS,ELEVPROB,ACQRR,OVALLSAT
ResponseID,,,,,,,,,,
103,yes,5,5,5,5,5,5,5,5,5
151,no,2,3,3,4,3,3,4,3,3
162,no,3,3,3,3,3,3,3,3,4
202,no,3,3,3,3,4,2,4,4,4
234,no,4,5,5,4,5,5,5,5,5


In [5]:
#ensure this dummy variable only contains yes and no
design_df['IPT_DUMMY'].value_counts()

no     135
yes     60
Name: IPT_DUMMY, dtype: int64

In [6]:
design_df[[x for x in design_df.columns if x != 'IPT_DUMMY']] = design_df[[x for x in design_df.columns if x != 'IPT_DUMMY']].astype(int)
#convert IPT_DUMMY to boolean
design_df['IPT_DUMMY'] = design_df.IPT_DUMMY == 'yes'
#get the number of ResponseIDs here for a future assertion
n_responses = len(design_df.index.unique())
design_df.index.name = None
design_df = design_df.reset_index()
design_df = design_df.drop(labels = 'index', axis = 1)
design_df.head()

Question,IPT_DUMMY,ACQMILE,SCHCHG,ACQASSIST,INDENG,ACQCOM,RESISS,ELEVPROB,ACQRR,OVALLSAT
0,True,5,5,5,5,5,5,5,5,5
1,False,2,3,3,4,3,3,4,3,3
2,False,3,3,3,3,3,3,3,3,4
3,False,3,3,3,3,4,2,4,4,4
4,False,4,5,5,4,5,5,5,5,5


In [7]:
#ensure this dummy variable only contains yes and no
design_df['IPT_DUMMY'].value_counts()

False    135
True      60
Name: IPT_DUMMY, dtype: int64

In [8]:
design_df.isnull().sum()

Question
IPT_DUMMY    0
ACQMILE      0
SCHCHG       0
ACQASSIST    0
INDENG       0
ACQCOM       0
RESISS       0
ELEVPROB     0
ACQRR        0
OVALLSAT     0
dtype: int64

In [9]:
#ensure no dupe responses
n_responses == design_df.shape[0]

True

# Exploratory Data Analysis
Summary stats for the variables.

In [10]:
pandas_profiling.ProfileReport(design_df)

Number of variables,10
Number of observations,195
Total Missing (%),0.0%
Total size in memory,14.0 KiB
Average record size in memory,73.4 B
Numeric,9
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,0


## Interpretations
All of the variables are ordinal, suggesting that an ordered logit model be used (would have to inspect proportional odds assumption though).


Treating this as an OLS instead of MLE problem, a few issues stand out:

1. **None of the ordinal predictors have values of $0$.** This means we'll likely want to **center the data**, especially since we'll be using `IPT_DUMMY` as an interaction term. Centering won’t actually change what the model means, but it can make the results more interpretable. Namely, the intercept will show the average `OVALLSAT` for someone in the reference group (IPT==0) with average values for all the predictors. To center, we can subtract the mean of a column from that column (exlcuding the dependent variable and the categorical variable). We could subtract some other value, but the key is to have a score of $0$ on the IV correspond to something that is substantively
interesting, rather than have it be a value that could not (or at least does not) actually occur in the data. This [short read](https://www3.nd.edu/~rwilliam/stats2/l53.pdf) explains further.

2. **The predictors are all ordinal and not necessarily continuous.** Using OLS, we'll be making the assumption that there are "equal" intervals between the likert values in each IV.

3. **All of the predictors are left skewed, with high correlations.** As a result there are very high correlations between all pairwise combinations of predictors. This is problematic because highly correlated exogenous predictors can affect the stability of our coefficient estimates as we make minor changes to model specification.

4. **There are $102$ duplicate observations in this data set.** Duplicates can bias the estimated coefficients and standard errors, with the extent of this bias increasing with doublets, triplets, qunituplets, etc. Weighting the duplicate records by the inverse of their multiplicity, or dropping superfluous duplicates, could reduce the risk of obtaining biased estimates. However, this will likely overestimate standard errors, reducing the statistical power of estimates of the estimates. We could us a Monte Carlo simulation to investigate how various numbers and patterns of duplicate records affect the risk of obtaining biased estimates (c.f. [this](https://mpra.ub.uni-muenchen.de/69064/1/MPRA_paper_69064.pdf)).

# OLS

The OLS model is going to have a dummy-encoded predictor (`IPT_DUMMY`), which will make the intercept the mean of `OVALLSAT` for the reference category (i.e. the category numbered 0, meaning not in an IPT). 

However, $0$ will not be a meaningful value for any predictor within the data set. As such, we need to center the data so that we can more easily interpret the intercept. We'll perform the analysis with and without centering to see the effect.

In [11]:
ols = smf.ols('OVALLSAT ~ center(SCHCHG) + center(ACQASSIST) + center(INDENG) + center(RESISS) \
               + center(ELEVPROB) + center(ACQRR) + center(ACQMILE)*IPT_DUMMY + center(ACQCOM)*IPT_DUMMY',
              data = design_df).fit()

In [12]:
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:               OVALLSAT   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                     99.78
Date:                Mon, 22 Apr 2019   Prob (F-statistic):           3.57e-71
Time:                        15:00:01   Log-Likelihood:                -89.138
No. Observations:                 195   AIC:                             202.3
Df Residuals:                     183   BIC:                             241.6
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

## Interpretation

### `IPT_DUMMY` and Intercept

>Since we have the dummy variable `IPT_DUMMY` in the model, the `Intercept` of $4.3426$ tells us the mean value of `OVALLSAT` for the reference group when all other predictors are their average value. 

>The mean value of `OVALLSAT` for the comparison group (being in an IPT) is the intercept ($4.3426$) plus the coefficient for `C(IPT_DUMMY)[T.1]` ($-0.0007$). But since the p-values are very high, we shouldn't put much stock in any deeper meanings. The result is basically looking at the following groupby result. So, instead, we'll look at the interactions.

In [13]:
design_df.groupby(by='IPT_DUMMY')['OVALLSAT'].mean()

IPT_DUMMY
False    4.362963
True     4.266667
Name: OVALLSAT, dtype: float64

### `IPT_DUMMY` Interactions
Since we have interactions, we are primarily interested in their significance, rather than the significance of the terms used to compute them. For example, looking at the `ACQMILE` coefficient alone only tells us about the difference between `IPT=1` and `IPT=0` at a specific point, i.e. when ACQMILE is its average value (`ACQMILE = 0` since we centered).

Looking at the interactins, we can see for `ACQMILE` that:

 - When IPT_DUMMY = 0, `ACQMILE` coefficient is -0.0374. 
 - When IPT_DUMMY = 1, `ACQMILE` coefficient is 0.1525 (-0.0374 + 0.1899)


Looking at the interactins, we can see for `ACQCOM` that:
 - When IPT_DUMMY = 0, `ACQCOM` coefficient is 0.3863
 - When IPT_DUMMY = 1, `ACQCOM` coefficient is 0.1694 (0.3863 + -0.2169)

# OLS Diagnositcs

In order to deterimine if we have a best linear unbiased estimator (BLUE), we need to test the assumptions made by the Gauss–Markov theorem.

## 1. Linearity
The dependent variable is assumed to be a linear function of the variables specified in the model. As such, our model  specification must be linear in its parameters. Note that this does not mean that there must be a linear relationship between the independent and dependent variables.

There are a few non-linearity tests we can run with `statsmodels`. We'll use the [Rainbow Test](https://www.tandfonline.com/doi/abs/10.1080/03610928208828423) for linearity, which has a null hypothesis that the regression is correctly modeled as linear. The basic idea of the Rainbow test is that even if the true relationship is non-linear, a good linear fit can be achieved on a subsample in the "middle" of the data. It is based on comparing a fit over low leverage points with a fit over the entire set of data. The null hypothesis is rejected whenever the overall fit is significantly worse than the fit for the subsample. The test statistic under $H_0$ follows an $F$ distribution with `n parameter` degrees of freedom.

In [15]:
alpha = .05
fstat, p_value = linear_rainbow(ols)
print(fstat, p_value)
if p_value < .05:
    print('This assumption is violated.')
else:
    print('This assumption holds.')

1.5035716578504208 0.027454183347474088
This assumption is violated.


## 2. Strict exogeneity
The conditional mean should be zero. In other words, the distribution of error terms has zero mean and doesn’t depend on the independent variables. This can commonly occur with [omitted variable bias](https://en.wikipedia.org/wiki/Omitted-variable_bias), as the omitted independent variable is correlated with both the dependent variable and one or more of the included independent variables, causing a spurious model specification. (If this is suspected to be the case, we could use two-stage least squares (2SLS) regression, which requires finding an instrumental variable).

We can calculate the correlation between our error and X's like this:

In [19]:
error_term = ols.resid
for col in design_df.columns:
    if col == 'const' or col == 'OVALLSAT':
        #constant
        continue
    _X = design_df[col]
    r, p_value = pearsonr(error_term, _X)
    print(f"{col}: {r}, {p_value}")

IPT_DUMMY: 2.5619200387966254e-15, 1.0
ACQMILE: -2.6683353985726322e-15, 1.0
SCHCHG: -2.2196535873730315e-15, 1.0
ACQASSIST: -2.4275606629733105e-15, 1.0
INDENG: -2.0575754673116167e-15, 1.0
ACQCOM: -2.026235313966836e-15, 1.0
RESISS: -2.3057205227195264e-15, 1.0
ELEVPROB: -2.3360559882174034e-15, 1.0
ACQRR: -2.043086674842592e-15, 1.0


## 3. No multi-collinearity (or perfect collinearity)
Multicollinearity (so long as it is not "perfect") can be present and still result in an unbiased estimate. Even so, the estimates might be less precise and highly sensitive to particular sets of data. Multicollinearity can be detected from condition number or the variance inflation factor, among other tests. We'll use these and then simulate the exlcusion of influential observations to see the effect on the model's parameters.

> #### Variance Inflation Factor


In [20]:
X = design_df[cols.values()].drop(labels = ['IPT_DUMMY', 'OVALLSAT'], axis = 1)
X_cols = list(X.columns)
X_cols.insert(0,'const')
X = X.values
#add columns of ones to represent a contsant for the OLS that statsmodels implements within variance_inflation_factor
X = sm.add_constant(X)
vifs = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
for col, vif in zip(X_cols, vifs):
    print(f"{col} has vif of {vif}.")

const has vif of 27.343666796441738.
ACQMILE has vif of 3.2641936181629725.
SCHCHG has vif of 5.743811470687627.
ACQASSIST has vif of 5.603519527149776.
INDENG has vif of 3.7975090136261804.
ACQCOM has vif of 6.5231520487462875.
RESISS has vif of 8.489678332708639.
ELEVPROB has vif of 5.83418496885111.
ACQRR has vif of 6.107235524337425.


>High VIFs are only a problem for the variables that are collinear. It increases the standard errors of their coefficients, and it may make those coefficients unstable in several ways. But so long as the collinear variables are only used as control variables, and they are not collinear with our variables of interest, there’s no problem. The coefficients of the variables of interest are not affected, and the performance of the control variables as controls is not impaired.

> #### Condition number
Another way to assess multicollinearity is to compute the [condition number](https://en.wikipedia.org/wiki/Condition_number). 

Very roughly, the condition number is the rate at which the solution, $x$ , will change with respect to a change in $b$. Thus, if the condition number is large, even a small error in $b$  may cause a large error in $x$.

In [21]:
ols.condition_number

12.16194532760197

> #### Influential observations
We can also use the idea of influential obeservations. Basically, an influential point is one whose deletion has a large effect on the parameter estimates of the regression model. A formal method is  using [DFBETA](https://en.wikipedia.org/wiki/Influential_observation) -- a standardized measure of how much each coefficient changes when that observation is left out. In general we may consider DBETA in absolute value greater than $2/\sqrt{N}$ to be influential observations. Another cut-off is to look for observations with a value greater than 1.00. Here cutoff means, “this observation could be overly influential on the estimated coefficient.” 

## 4. Spherical Errors
The error term (residuals) should have uniform variance (homoscedasticity) and no serial dependence (i.e. autocorrelation).

### Heteroscedasticity
These heteroscedasticity tests test the null hypothesis that all observations have the same error variance, i.e. errors are homoscedastic. The tests differ in which kind of heteroscedasticity is considered as alternative hypothesis. They also vary in the power of the test for different types of heteroscedasticity.

 - `het_breuschpagan`: Lagrange Multiplier Heteroscedasticity Test by Breusch-Pagan
 - `het_white`: Lagrange Multiplier Heteroscedasticity Test by White
 - `het_goldfeldquandt`: test whether variance is the same in 2 subsamples

# Weighted Least Squares
Due to the limitations identified with the OLS models above, we'll use weighted least squares to correct for the heteroskedasticity.

Supposedely the variance is proportional to `INDENG*INDENG` ($x$), so that $Var(y_{i}) = x_i\sigma^2$ and $w_i =1/ x_i$.

In [50]:
#In statsmodels, the weights are presumed to be (proportional to) the inverse of the variance of the observations. 
#If you supply weights = 1/W then the variables are pre-multiplied by 1/sqrt(W)
weights = 1/design_df['INDENG']**2

wls = smf.wls('OVALLSAT ~ SCHCHG + ACQASSIST + INDENG + RESISS \
               + ELEVPROB + ACQRR + ACQCOM*IPT_DUMMY + ACQMILE*IPT_DUMMY', 
              weights = weights,
              data = design_df).fit()
print(wls.summary())

                            WLS Regression Results                            
Dep. Variable:               OVALLSAT   R-squared:                       0.894
Model:                            WLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     140.2
Date:                Mon, 22 Apr 2019   Prob (F-statistic):           6.20e-83
Time:                        15:16:35   Log-Likelihood:                -183.66
No. Observations:                 195   AIC:                             391.3
Df Residuals:                     183   BIC:                             430.6
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             